## load package

In [199]:
# load package
# string 
import re

# math
import pandas as pd
import numpy as np
import scipy as sp
import random

# sys
import sys
import os
import time
import warnings

# date
from datetime import datetime, timedelta

# machine learning
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, gaussian_process, discriminant_analysis
from xgboost import XGBClassifier

# model utils
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection 
from sklearn import model_selection
from sklearn import metrics

# plot
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix #??
# = show plots in Jupyter Notebook browser
%matplotlib inline 
mpl.style.use('ggplot') #??
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8 #??

# show all columns
from IPython.display import display
pd.options.display.max_columns = None

# memory manage
import gc

# logging
import logging 

# other
import tqdm as tqdm

# self define
sys.path.append('../../utils/')

In [206]:
logging.basicConfig(level = logging.INFO, format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler = logging.FileHandler('../log/extract_features.log')
handler.setLevel(logging.INFO)
formater = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formater)
logger = logging.getLogger(__name__)
logger.addHandler(handler)
logger.info('This is a log info')

2018-10-30 09:33:38,585 - __main__ - INFO - This is a log info


In [2]:
# read files
lb_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2014.csv', sep = ';')
lb_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2015.csv', sep = ';')
lb_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2016.csv', sep = ';')
lb_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2017.csv', sep = ';')
lb_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/leonberg_werkstattposten_2018.csv', sep = ';')
std_wsp_2014 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2014.csv', sep = ';')
std_wsp_2015 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2015.csv', sep = ';')
std_wsp_2016 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2016.csv', sep = ';')
std_wsp_2017 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2017.csv', sep = ';')
std_wsp_2018 = pd.read_csv('../data/Autohaus_weeber/weil_der_stadt_werkstattposten_2018.csv', sep = ';')
# cat
d1 = lb_wsp_2014.copy()
d2 = lb_wsp_2015.copy()
d3 = lb_wsp_2016.copy()
d4 = lb_wsp_2017.copy()
d5 = lb_wsp_2018.copy()

d6 = std_wsp_2014.copy()
d7 = std_wsp_2015.copy()
d8 = std_wsp_2016.copy()
d9 = std_wsp_2017.copy()
d10 = std_wsp_2018.copy()

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# for training we use data from 14 - 17 in lb_wsp
train1 = pd.concat([d1,d2,d3,d4,d5], 0)
train2 = pd.concat([d6,d7,d8,d9,d10], 0)
train1['Autohaus'] = 'leonberg'
train2['Autohaus'] = 'weil'
#train = pd.concat([train1, train2], 0)
#train = train1.copy()

In [4]:
# because there exists reused Auftragsnummer in different Autohaus station. So we add some sign to the Auftragsnummer
# in each Autohaus station
train1['Auftragsnummer'] = 'A' + train1['Auftragsnummer']
train2['Auftragsnummer'] = 'B' + train2['Auftragsnummer']
train = pd.concat([train1, train2], 0)

## extract Teile-Gruppe
An 4 . Stelle der Teile-Nr ist die Baugruppe 
- 1 - Motor
- 2 - Kraftstoff, Abgas, Kühlung
- 3 - Getrieb
- 4 - Vorderachse, Lenkung
- 5 - Hinterachse
- 6 - Räder
- 7 - Hebelwerk
- 8 - Karosserie
- 9 - Elektrik
- 0 - Zubehör, Infotainment, Sonstiges

#### remove the outlier in the Teile-Nr
- na wert
- Teile-Nr, which length smaller than 4
- doesn't contain number??
- doesn't contain adjoining number 

Question: The Gruppe-Nr can also in the 5. stelle<br>
#df[df['Teile-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', x[3])))]['Teile-Nr']<br>
the command show the Teile-Nr, which type are not number. We can see that many Teile-Nr doesn't contrain number; some Teile-Nr contain number, however there are not in the 4. stelle bu in the 5. oder 6. stelle. Therefore we suggest that the first coming adjoining nummer in the Teile-Nr may be the gruppe-Nr.<br>
We extract the gruppe-Nr according to this idea.<br>
This phase takes train data as input and output Dataframe df

In [5]:
# remove the outlier in the Teile-Nr
# remove the na wert
df = train[train['Teile-Nr'].isna().map(lambda x: not x)]
# remove the value short value
df = df[df['Teile-Nr'].map(lambda x: False if len(x) < 4 else True)]
# remove the value that doesn't contain number: 30593 of 593527 in train1(5%)
df = df[df['Teile-Nr'].map(lambda x: True if re.search('\d', x) else False)]
# remove the value that doesn't contrain adjoining number(min 2 number): 594 of 562934 in train1(0.1%)
df = df[df['Teile-Nr'].map(lambda x: True if re.search('\d\d', x) else False)]

In [6]:
# use findall instead of search, because, the df here should contain adjoining number, otherwise it's wrong
df['Gruppe-Nr'] = df['Teile-Nr'].map(lambda x: re.findall('\d\d', x)[0][0])

In [7]:
df['Gruppe-Nr'].value_counts()

0    930093
1    281620
9    241658
3     46656
2     37419
6     23866
8     21086
5     19447
4     14385
7      6041
Name: Gruppe-Nr, dtype: int64

In [8]:
# clean the memory
del train1, train2, lb_wsp_2014, lb_wsp_2015, lb_wsp_2016, lb_wsp_2017, lb_wsp_2018
del std_wsp_2014, std_wsp_2015, std_wsp_2016, std_wsp_2017, std_wsp_2018
del d1, d2, d3, d4, d5, d6, d7, d8, d9, d10
gc.collect()

91

In [10]:
#

## remove the confused data
- For the same Auftragsnummer, exists more than one value in the other attribute.
- wierd Auftragsnummer: 107K, 77KW
- Fahrgestellnummer is Na
- Fahrgestellnummer that only contrain number.
- In Auftragsdatum, the day of the datum is larger than 31
- Wierd Auftragsdatum, these datum is shorter than 10 Ziffer.
- Wierd Auftragsdatum, these datum is smaller than 2013

In [11]:
# not only see that data with gruppe-nr == 9, 241658 items
# the target of next coming can also for the other thing, not the eletric
# df = df[df['Gruppe-Nr'] == '9']

In [9]:
# choose the features, which may be essentail to the model
# We will take care of the 'AW-Nr' and 'KM-Stand' in the other place.
# there are 98256 unique items in cf, more than 90423 items in af, which means they may create NA value after merging
#cf = train[['Auftragsnummer', 'AW-Nr', 'Markencode', 'Lagerortcode', 'Adressanredecode', 'Motorcode', 'Fahrzeugmodellnummer', 'Modell', 'Typ',
#        'Getriebecode', 'Gewicht', 'Leistung (KW)', 'Fahrgestellnummer']]
cf = df.copy()

In [10]:
# drop duplicate
cf = cf.drop_duplicates()

In [11]:
# find out the confused data. For the same Auftragsnummer, exists more than one value in the other attribute.
# here is AWSAU310019, BWSAU386471, BWSAU435051
cf = cf.drop(cf[cf['Auftragsnummer'] == 'AWSAU310019'].index, axis= 0) # 8 items
cf = cf.drop(cf[cf['Auftragsnummer'] == 'BWSAU386471'].index, axis= 0) # 3 items
cf = cf.drop(cf[cf['Auftragsnummer'] == 'BWSAU435051'].index, axis= 0) # 2 items
cf = cf.drop(cf[cf['Auftragsnummer'] == 'BWSAU271939'].index, axis= 0) # ? items

In [12]:
# remove wierd auftragsnummer
cf = cf.drop(cf[cf['Auftragsnummer'] == '103K'].index) #
cf = cf.drop(cf[cf['Auftragsnummer'] == '77KW'].index) #

In [13]:
# remove na value in Fahrgestellnummer
cf = cf.drop(cf[cf['Fahrgestellnummer'].isna()].index, axis = 0) #

In [14]:
# remove the Fahrgestellnumer that only contain number, not tested!!!!
cf = cf.drop(cf[cf['Fahrgestellnummer'].map(lambda x: False if re.search('[a-zA-Z]', x) else True)].index, axis = 0)

In [15]:
# remove the items, which day of the Auftragsdatum lareger than 31, not tested!!!!
cf = cf.drop(cf[cf['Auftragsdatum'].map(lambda x: int(x[0:2])) > 31].index, axis = 0)

In [16]:
# remove the items, which length of the Auftragsdatum shorter than 10, not tested!!!!
cf = cf.drop(cf[cf['Auftragsdatum'].map(lambda x:True if len(x) < 10 else False)].index, axis = 0)

In [17]:
# remove the items, which year of the Auftragsdatum smaller than 2013, not tested!!!!
cf = cf.drop(cf[cf['Auftragsdatum'].map(lambda x: int(x[6:]) < 2013)].index, axis = 0)

In [21]:
#len(cf)

In [18]:
df = cf
del cf
gc.collect()

141

In [ ]:
#check_to_1(tmp, 'Auftragsnummer', 'Markencode')

In [ ]:
#check_to_1(tmp, 'Auftragsnummer', 'Lagerortcode')

In [18]:
#check_to_1(tmp, 'Auftragsnummer', 'Adressanredecode')

245023
21
BWSAU435051	['Herr', 'Firma']


In [19]:
#check_to_1(tmp, 'Auftragsnummer', 'Motorcode')

245023
1267
AWSAU310019	['DDYA', 'AKQ']
BWSAU435051	['CFHC', 'CRLB']


In [20]:
#check_to_1(tmp, 'Auftragsnummer', 'Fahrzeugmodellnummer')

245023
7134
AWSAU310019	['BV527Z', '6X10Z4']
BWSAU386471	['NJ33E4', '']
BWSAU435051	['1T33A3', '3V5455']


In [21]:
#check_to_1(tmp, 'Auftragsnummer', 'Modell')

245023
10538
AWSAU310019	['Golf Variant Trendline', 'VW Lupo > 99']
BWSAU386471	['Fabia Ambition 1,0l MPI 55 kW', '']
BWSAU435051	['Touran Comfortline 2,0 l TDI', 'Superb Combi Style TDI 110 kW']


In [22]:
#check_to_1(tmp, 'Auftragsnummer', 'Typ')

245023
557
AWSAU310019	['X0A', '']
BWSAU435051	['X0A', '']


In [23]:
#check_to_1(tmp, 'Auftragsnummer', 'Getriebecode')

245023
2799
AWSAU310019	['GKB', 'DKF']
BWSAU435051	['PPY', 'QFZ']


In [24]:
#check_to_1(tmp, 'Auftragsnummer', 'Gewicht')

245023
53
BWSAU386471	['0,00', '']


In [25]:
#check_to_1(tmp, 'Auftragsnummer', 'Leistung (KW)')

245023
143
AWSAU310019	['85.0', '0.0']
BWSAU386471	['0.0', '']
BWSAU435051	['103.0', '0.0']


In [26]:
#check_to_1(tmp, 'Auftragsnummer', 'Fahrgestellnummer')

245023
70747
AWSAU310019	['WVWZZZAUZJP531831', 'WVWZZZ6XZXW035409']
BWSAU386471	['TMBEB6NJ7JZ032235', '']
BWSAU435051	['WVGZZZ1TZEW001359', 'TMBJH7NP5J7507173']


In [100]:
#check_to_1(train, 'Auftragsnummer', 'KM-Stand')

292528
124554


In [102]:
#check_to_1(train, 'Auftragsnummer', 'Erstzulassungsdatum')

292528
6613
AWSAU310019	['27.09.2017', '01.03.1999']
BWSAU271939	['14.04.2009', '22.06.2012']
BWSAU386471	['18.09.2017', '']
BWSAU435051	['03.06.2013', '21.07.2017']


## Get Auftrag table for Gruppe-Nr

In [19]:
gn = df.copy()

In [26]:
gn = toAuftragTable(gn, 'Gruppe-Nr', 'Auftragsnummer') # number: 245018

Falls Null date exist, drop these dates directly


In [27]:
len(gn)

244077

## extract feature: total AW-Nr
extract the feature total AW and combine aw with gruppe-nr auftragstable to form a new Dataframe agn. <br>
combine the attribute in df to the agn

In [28]:
aw = train.copy() # can't use df here, because df delete all the na value in Teile-Nr

In [29]:
# remove the items, that contain ziffer in the AW-Nr, set data type to int
aw['AW-Nr'][aw['AW-Nr'].isnull()] = 0 # set null value to 0
aw['AW-Nr'][aw['AW-Nr'] == '99999999'] = 0 # set 99999999 to 0
aw['AW-Nr'] = aw['AW-Nr'].map(lambda x: re.sub(' ', '', str(x)))# remove space between the number
del_index = aw[aw['AW-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', str(x))))].index # remove fake data
aw = aw.drop(del_index) 
aw['AW-Nr'] = aw['AW-Nr'].astype('float')

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:
# to auftrag table, get the feature: total AW-Nr for each auftrag
tmp = aw[['AW-Nr', 'Auftragsnummer']]
ag = tmp.groupby('Auftragsnummer', as_index = False)
out = []
for name, gruppe in ag:
    out.append(pd.DataFrame({'Auftragsnummer': [gruppe['Auftragsnummer'].iloc[0]], 'total AW': [gruppe['AW-Nr'].sum()]}))
aw = pd.concat(out).reset_index()[['Auftragsnummer', 'total AW']] # 缺分段
# drop the wierd Auftragsnummer
aw = aw.drop(aw[aw['Auftragsnummer'] == '103K'].index)
aw = aw.drop(aw[aw['Auftragsnummer'] == '77KW'].index)

In [31]:
# the lengths of aw and gn are different, because in some auftrag, the customer do not buy anything. 
# These part of auftrags are not included in gn, but gn is not a subset of aw. 
# because some AW-Nr in aw are wrong, we have deleted these items in aw. 
# Therefore, if we want to merge aw and gn, we have to use the method inner
agn = pd.merge(aw, gn, how = 'inner', on = 'Auftragsnummer')

## extract feature: num_act
not tested jet<br>
make sure that, the length of the output, shoube be the same as the length of agn

In [32]:
aw = train.copy()
# remove the items, that contain ziffer in the AW-Nr, set data type to int
aw['AW-Nr'][aw['AW-Nr'].isnull()] = 0 # set null value to 0
aw['AW-Nr'][aw['AW-Nr'] == '99999999'] = 0 # set 99999999 to 0
aw['AW-Nr'] = aw['AW-Nr'].map(lambda x: re.sub(' ', '', str(x)))# remove space between the number
del_index = aw[aw['AW-Nr'].map(lambda x: bool(re.search('[a-zA-Z]', str(x))))].index # remove fake data
aw = aw.drop(del_index) 
aw['AW-Nr'] = aw['AW-Nr'].astype('float')
# to auftrag table, get the feature: total AW-Nr for each auftrag
tmp = aw[['AW-Nr', 'Auftragsnummer']]
ag = tmp.groupby('Auftragsnummer', as_index = False)
out = []
for name, gruppe in ag:
    out.append(pd.DataFrame({'Auftragsnummer': [gruppe['Auftragsnummer'].iloc[0]], 'num_act': [len(gruppe)]}))
aw = pd.concat(out).reset_index()[['Auftragsnummer', 'num_act']] # 缺分段
# drop the wierd Auftragsnummer
aw = aw.drop(aw[aw['Auftragsnummer'] == '103K'].index)
aw = aw.drop(aw[aw['Auftragsnummer'] == '77KW'].index)

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [33]:
#agn = pd.merge(aw, agn, how = 'inner', on = 'Auftragsnummer')
agn = pd.merge(aw, agn, how = 'inner', on = 'Auftragsnummer')

In [34]:
agn.head()

,Auftragsnummer,num_act,total AW,Gruppe-Nr
0,AWSAU195981,31,1.526062e+09,3;0
1,AWSAU198256,31,6.398138e+08,1;9;0
2,AWSAU200129,14,3.832960e+08,0;1;6;9
3,AWSAU200223,60,1.841819e+09,3;6;9;1;0;5
4,AWSAU200258,23,5.367113e+08,1;9;0


In [35]:
#train[train['Auftragsnummer'] == 'AWSAU195981']

## extract feature: num_teile

In [36]:
agn['num_teile'] = agn['Gruppe-Nr'].map(lambda x: len(x.split(';')))

In [37]:
agn[['total AW', 'num_act']].corr() 
# why the correlation is so small??

,total AW,num_act
total AW,1.000000,0.584288
num_act,0.584288,1.000000


In [38]:
del aw, gn
gc.collect()

112

## combine agn with df

In [39]:
tmp = df[['Auftragsnummer', 'KM-Stand','Markencode', 'Lagerortcode', 'Auftragsdatum', 'Adressanredecode', 'Motorcode', 'Fahrzeugmodellnummer', 'Modell', 'Typ', 
          'Getriebecode', 'Gewicht', 'Leistung (KW)', 'Fahrgestellnummer', 'Erstzulassungsdatum']]
tmp = tmp.drop_duplicates()
tmp = pd.merge(agn, tmp, how = 'inner', on = 'Auftragsnummer') # 244608

In [40]:
df = tmp
del tmp
gc.collect()

41

In [98]:
#TODO
# calculate the length of aw and original df
# i suggest that, because i delete all the na value in Teile-Nr, the number of items in original df is smaller
# than the number in aw.
# the result is, after mergin, there are many na value in some attribute, 
# i want to see the number of distance.
# or may be i can merge the data in the other direction: 'right'
# but mit direction right, there exist also 600 null value but in total AN, the reason may be
# that i delete some fake data in aw.
# the problem here is, should i maintain the na value in total AW, other just remove all of them?

## extract features: month, day

In [41]:
md = df.copy()
# remove the outlier in the data
#fake = md[md['Auftragsdatum'].map(lambda x:True if re.search('[a-zA-Z]', x) else False)]
#md = md.drop(fake.index, axis=0)

In [42]:
md['year'] = md['Auftragsdatum'].map(lambda x: x[6:])
md['month'] = md['Auftragsdatum'].map(lambda x: x[3:5])
md['day'] = md['Auftragsdatum'].map(lambda x: x[0:2])

In [43]:
df = md
del md
gc.collect()

28

In [44]:
df['num_teile'].value_counts()

1     99287
3     66428
2     50047
4     20344
5      5373
6      1446
7       494
8       185
9        53
10       10
Name: num_teile, dtype: int64

## extract feature: age
not tested jet!!!<br>
age_auto = auftragsdatum - Erstzulassungsdatum<br>
age_autohaus = auftragsdatum - 2013

In [45]:
#tmp = train[['Auftragsdatum', 'Erstzulassungsdatum']]
tmp = df.copy()

In [46]:
tmp['Auftragsdatum'] = pd.to_datetime(tmp['Auftragsdatum'], dayfirst=True)
tmp['Erstzulassungsdatum'] = pd.to_datetime(tmp['Erstzulassungsdatum'], dayfirst=True)
tmp['age_auto'] = tmp['Auftragsdatum'] - tmp['Erstzulassungsdatum']

In [47]:
tmp['age_autohaus'] = tmp['Auftragsdatum'].map(lambda x: x.year) - 2013

In [48]:
df = tmp
del tmp
gc.collect()

14

In [49]:
df.head(10)

,Auftragsnummer,num_act,total AW,Gruppe-Nr,num_teile,KM-Stand,Markencode,Lagerortcode,Auftragsdatum,Adressanredecode,Motorcode,Fahrzeugmodellnummer,Modell,Typ,Getriebecode,Gewicht,Leistung (KW),Fahrgestellnummer,Erstzulassungsdatum,year,month,day,age_auto,age_autohaus
0,AWSAU195981,31,1.526062e+09,3;0,2,"38110,00",VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",2014-01-13,Firma,CRBC,5G131X,Golf VII 2.0 TDI CL BMT,X0A,PFL,"0,00",110.0,WVWZZZAUZDP084627,2013-05-17,2014,01,13,241 days,1
1,AWSAU198256,31,6.398138e+08,1;9;0,3,"59660,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-16,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,2012-07-27,2014,01,16,538 days,1
2,AWSAU200129,14,3.832960e+08,0;1;6;9,4,"227777,00",VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",2014-01-09,Herr,CBAB,5K143M,"Golf Highline 2,0 l",X0A,LQV,"0,00",0.0,WVWZZZ1KZAW187314,2009-11-25,2014,01,09,1506 days,1
3,AWSAU200223,60,1.841819e+09,3;6;9;1;0;5,6,"65174,00",VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",2014-01-07,Herr,AXR,1J5134,VW Golf IV > 98 / Variant > 99,NaN,EUH,"0,00",0.0,WVWZZZ1JZ2W511716,2002-02-15,2014,01,07,4344 days,1
4,AWSAU200258,23,5.367113e+08,1;9;0,3,"212110,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Firma,CAGA,8K50QC,A4 Avant 2.0 TDI Attraction,X0A,LLN,"0,00",105.0,WAUZZZ8K5AA164327,2010-05-06,2014,01,02,1337 days,1
5,AWSAU200259,9,1.862719e+08,9;0,2,"91895,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,ALT,8E20B4,A4 Limousine 2.0 R4 96 M5S,NaN,GBM,"0,00",0.0,WAUZZZ8E13A363100,2003-04-24,2014,01,02,3906 days,1
6,AWSAU200260,11,3.962096e+08,0,1,"122325,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,CAYC,8P1AE4,Audi A3 Attraction 3-Türer 1.6,X0A,LUB,"0,00",0.0,WAUZZZ8PXAA157479,2010-04-29,2014,01,02,1344 days,1
7,AWSAU200263,36,6.978901e+08,3;1;9;0,4,"112386,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,ASB,4F20LL,Audi A6 Limousine 3.0 TDI (DPF,X0A,KGX,"0,00",0.0,WAUZZZ4F28N185258,2008-10-04,2014,01,02,1916 days,1
8,AWSAU200264,12,1.656055e+08,3;8;0,3,"42783,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,CLAB,4G50NH,A6 Avant 3.0 TDI,X0A,NKP,"0,00",150.0,WAUZZZ4G4CN101295,2012-02-21,2014,01,02,681 days,1
9,AWSAU200269,8,1.641055e+08,3;0,2,"180999,00",AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",2014-01-02,Herr,BPP,8ED0YH,Audi A4 Avant 2.7 TDI(DPF),X0A,JBB,"0,00",0.0,WAUZZZ8E07A224307,2007-03-29,2014,01,02,2471 days,1


## transfer the type of km stand to float

In [50]:
df['KM-Stand'] = df['KM-Stand'].map(lambda x: float(re.sub(',', '.', x)))

## extract feature: KM age

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243667 entries, 0 to 243666
Data columns (total 24 columns):
Auftragsnummer          243667 non-null object
num_act                 243667 non-null int64
total AW                243667 non-null float64
Gruppe-Nr               243667 non-null object
num_teile               243667 non-null int64
KM-Stand                243667 non-null object
Markencode              243468 non-null object
Lagerortcode            243667 non-null object
Auftragsdatum           243667 non-null datetime64[ns]
Adressanredecode        241586 non-null object
Motorcode               235550 non-null object
Fahrzeugmodellnummer    243477 non-null object
Modell                  242305 non-null object
Typ                     162462 non-null object
Getriebecode            234752 non-null object
Gewicht                 243461 non-null object
Leistung (KW)           243461 non-null float64
Fahrgestellnummer       243667 non-null object
Erstzulassungsdatum     243461 non-

## extract feature: time relative
- time distance since last come
- time distance to next come
- km stand distance since last come(correct the data first)
- km stand distance to next come
- is km stand to/from next/last come larger than threshold
- frequent in each year, in total
- (not jet)weather come in X month again
- time shift for num_act, total aw, Gruppe-Nr, num_teile
- repair age in year or in month
- repair frequent in last X month
- trend of frequent in the past
We sort the dataframe with ['Fahrgestellnummer', 'Auftragsdatum'], and calculate the datum distance betwenn the nearby items.<br>
One thing to be clear is that, the first distance for each Fahrgestellnummer is wrong, because we calculate it with the its Auftragsdatum and the Auftragsdatum of the other Fahrgestellnummer.<br>

In [51]:
td = df.copy()

In [52]:
td = td[['Fahrgestellnummer', 'Auftragsdatum', 'KM-Stand','num_act', 'total AW', 'num_teile', 'year']]
#1C8FYN8O63T585854 8

In [71]:
# get sample, to test the code
for name, group in td.groupby('Fahrgestellnummer', as_index = False):
    if name == '1C8FYN8O63T585854':
        tmp = group.reindex()
        print(tmp.columns)
        print(tmp.index)
        # time distance since/to last/next come
     #create feature: time distance from last coming

Index(['Fahrgestellnummer', 'Auftragsdatum', 'KM-Stand', 'num_act', 'total AW',
       'num_teile', 'year'],
      dtype='object')
Int64Index([103323, 108984, 112403, 114316, 125322, 159899, 171613, 193678], dtype='int64')


In [195]:
# code to extract new features, remmenber to remove counter
out = []
counter = 0
for name, group in td.groupby('Fahrgestellnummer', as_index = False):
    counter = counter + 1
    if counter > 10:
        break
    #if name == '1C8FYN8O63T585854':
    #    tmp = group.reindex()
    #    print(tmp.columns)
    #    print(tmp.index)
    #    # time distance since/to last/next come
    # create feature: time distance from last coming
    tmp = group
    if len(tmp) > 1:
        # create feature: time distance from last coming
        time_shift_forward = pd.concat([pd.Series(datetime(2010, 1, 1)), tmp['Auftragsdatum'][:-1]])
        time_shift_forward.index = tmp.index
        tmp['time_distance_from_last'] = tmp['Auftragsdatum'] - time_shift_forward
        # create feature: time distance to next coming
        time_distance_to_next = pd.concat([tmp['time_distance_from_last'][1:], pd.Series(np.NaN)])
        time_distance_to_next.index = tmp.index
        tmp['time_distance_to_next'] = time_distance_to_next
        # reset the first value and last value
        tmp.iloc[0, tmp.columns.get_loc('time_distance_from_last')] = np.NaN
        tmp.iloc[-1, tmp.columns.get_loc('time_distance_to_next')] = np.NaN
        # create feature: km stand distance from last coming
        km_shift_forward = pd.concat([pd.Series(0.0), tmp['KM-Stand'][:-1]])
        km_shift_forward.index = tmp.index
        tmp['km_distance_from_last'] = tmp['KM-Stand'] -km_shift_forward
        # create feature: km stand distance to next coming
        km_distance_to_next = pd.concat([tmp['km_distance_from_last'][1:], pd.Series(0.0)])
        km_distance_to_next.index = tmp.index
        tmp['km_distance_to_next'] = km_distance_to_next
        # reset the first value and last value
        tmp.iloc[0, tmp.columns.get_loc('km_distance_from_last')] = np.NaN
        tmp.iloc[-1, tmp.columns.get_loc('km_distance_to_next')] = np.NaN
        # create feature: frequent in each year
        year_candidates = ['2014', '2015', '2016', '2017', '2018']
        for year in year_candidates:
            tmp['frequent_in_'+year] = len(tmp[tmp['year'] == year])
        # create feature: time shift for num_act
        num_act_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_act'][:-1]])
        num_act_shift_forward.index = tmp.index
        tmp['num_act_shift1'] = num_act_shift_forward
        # create feature: time shift for num_teile
        num_teile_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_teile'][:-1]])
        num_teile_shift_forward.index = tmp.index
        tmp['num_teile_shift1'] = num_teile_shift_forward
        # create feature: time shift for total aw
        total_aw_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['total AW'][:-1]])
        total_aw_shift_forward.index = tmp.index
        tmp['total_aw_shift1'] = total_aw_shift_forward
        if len(tmp) > 2:
            num_act_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_act'][:-2]])
            num_act_shift_forward2.index = tmp.index
            tmp['num_act_shift2'] = num_act_shift_forward2
            num_teile_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_teile'][:-2]])
            num_teile_shift_forward2.index = tmp.index
            tmp['num_teile_shift2'] = num_teile_shift_forward2
            total_aw_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['total AW'][:-2]])
            total_aw_shift_forward2.index = tmp.index
            tmp['total_aw_shift2'] = total_aw_shift_forward2
        else:
            tmp['num_act_shift2'] = np.NaN
            tmp['num_teile_shift2'] = np.NaN
            tmp['total_aw_shift2'] = np.NaN
        # create feature: repair age in year or in month
        tmp['first_date'] = tmp.iloc[0, tmp.columns.get_loc('Auftragsdatum')]
        tmp['repair_age'] = tmp['Auftragsdatum'] - tmp['first_date']
        tmp = tmp.drop('first_date', axis=1)
        # create feature: repair frequent in last X month
        time_distance_shift1 = pd.concat([pd.Series(np.NaN), tmp['time_distance_from_last'][:-1]])
        time_distance_shift1.index = tmp.index
        tmp['time_distance_shift1'] = time_distance_shift1
        time_distance_shift2 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift1'][:-1]])
        time_distance_shift2.index = tmp.index
        tmp['time_distance_shift2'] = time_distance_shift2
        time_distance_shift3 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift2'][:-1]])
        time_distance_shift3.index = tmp.index
        tmp['time_distance_shift3'] = time_distance_shift3
        tmp['time_distance_from_last2'] = tmp['time_distance_from_last'] + tmp['time_distance_shift1']
        tmp['time_distance_from_last3'] = tmp['time_distance_from_last2'] + tmp['time_distance_shift2']
        tmp['time_distance_from_last4'] = tmp['time_distance_from_last3'] + tmp['time_distance_shift3']
        def get_frequent(x, d = 30):
            if x['time_distance_from_last4'] < timedelta(days = d):
                return 4
            elif x['time_distance_from_last3'] < timedelta(days = d):
                return 3
            elif x['time_distance_from_last2'] < timedelta(days = d):
                return 2
            elif x['time_distance_from_last'] < timedelta(days = d):
                return 1
            else:
                return 0
        tmp['frequent_in_last_1_month'] = tmp.apply(get_frequent, axis=1, args = [1*30])
        tmp['frequent_in_last_2_month'] = tmp.apply(get_frequent, axis=1, args = [2*30])
        tmp['frequent_in_last_3_month'] = tmp.apply(get_frequent, axis=1, args = [3*30])
        tmp['frequent_in_last_4_month'] = tmp.apply(get_frequent, axis=1, args = [4*30])
        tmp['frequent_in_last_5_month'] = tmp.apply(get_frequent, axis=1, args = [5*30])
        tmp['frequent_in_last_6_month'] = tmp.apply(get_frequent, axis=1, args = [6*30])
        # create feature: weather come in last X month
        tmp['weather_come_in_last_1_month'] = tmp['time_distance_from_last'] < timedelta(days = 1*30)
        tmp['weather_come_in_last_2_month'] = tmp['time_distance_from_last'] < timedelta(days = 2*30)
        tmp['weather_come_in_last_3_month'] = tmp['time_distance_from_last'] < timedelta(days = 3*30)
        tmp['weather_come_in_last_4_month'] = tmp['time_distance_from_last'] < timedelta(days = 4*30)
        tmp['weather_come_in_last_5_month'] = tmp['time_distance_from_last'] < timedelta(days = 5*30)
        tmp['weather_come_in_last_6_month'] = tmp['time_distance_from_last'] < timedelta(days = 6*30)
        # create feature: trend of frequent in the past
        def get_trend(x):
            xs = x[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']].tolist()
            xs = [x.days for x in xs if not pd.isnull(x)]
            if len(xs) < 2:
                return 0
            ys = range(1, len(xs) + 1)
            xm = sum(xs)/len(xs)
            ym = sum(ys)/len(ys)
            xy = [x*y for x, y in zip(xs, ys)]
            xx = [x*y for x, y in zip(xs, xs)]
            up = sum(xy) - len(xs)*xm*ym
            down = sum(xx) - len(xs)*xm*xm
            if down == 0:
                return 0
            return up*1.0/down
        tmp['trend_of_frequent_in_the_past'] = tmp.apply(get_trend, axis=1)
        # create feature: weather come in X month again
    else:
        # set features to NaN
        tmp['time_distance_from_last '] = np.NaN
        tmp['time_distance_to_next'] = np.NaN
        tmp['km_distance_from_last'] = np.NaN
        tmp['km_distance_to_next'] = np.NaN
        tmp['frequent_in_2014'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2014' else 0
        tmp['frequent_in_2015'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2015' else 0
        tmp['frequent_in_2016'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2016' else 0
        tmp['frequent_in_2017'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2017' else 0
        tmp['frequent_in_2018'] = 1 if tmp.iloc[0, tmp.columns.get_loc('year')] == '2018' else 0
        tmp['num_act_shift1'] = np.NaN
        tmp['num_teile_shift1'] = np.NaN
        tmp['total_aw_shift1'] = np.NaN
        tmp['num_act_shift2'] = np.NaN
        tmp['num_teile_shift2'] = np.NaN
        tmp['total_aw_shift2'] = np.NaN
        tmp['time_distance_shift1'] = np.NaN
        tmp['time_distance_shift2'] = np.NaN
        tmp['time_distance_shift3'] = np.NaN
        tmp['time_distance_from_last2'] = np.NaN
        tmp['time_distance_from_last3'] = np.NaN
        tmp['time_distance_from_last4'] = np.NaN
        tmp['weather_come_in_last_1_month'] = False
        tmp['weather_come_in_last_2_month'] = False
        tmp['weather_come_in_last_3_month'] = False
        tmp['weather_come_in_last_4_month'] = False
        tmp['weather_come_in_last_5_month'] = False
        tmp['weather_come_in_last_6_month'] = False
        tmp['trend_of_frequent_in_the_past'] = 0
    out.append(tmp)

# i don't known why the warning appear??? try to figure it out, it can save lots of time.

/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [196]:
df3 = pd.concat(out)

In [197]:
df3

,Auftragsdatum,Fahrgestellnummer,KM-Stand,frequent_in_2014,frequent_in_2015,frequent_in_2016,frequent_in_2017,frequent_in_2018,frequent_in_last_1_month,frequent_in_last_2_month,frequent_in_last_3_month,frequent_in_last_4_month,frequent_in_last_5_month,frequent_in_last_6_month,km_distance_from_last,km_distance_to_next,num_act,num_act_shift1,num_act_shift2,num_teile,num_teile_shift1,num_teile_shift2,repair_age,time_distance_from_last,time_distance_from_last,time_distance_from_last2,time_distance_from_last3,time_distance_from_last4,time_distance_shift1,time_distance_shift2,time_distance_shift3,time_distance_to_next,total AW,total_aw_shift1,total_aw_shift2,trend_of_frequent_in_the_past,weather_come_in_last_1_month,weather_come_in_last_2_month,weather_come_in_last_3_month,weather_come_in_last_4_month,weather_come_in_last_5_month,weather_come_in_last_6_month,year
7015,2014-05-20,------AU-00201251,266265.0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,NaN,2,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000,False,False,False,False,False,False,2014
129252,2015-08-04,159247535XXXXXXXX,87944.0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42,NaN,NaN,5,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.920609e+08,NaN,NaN,0.000000,False,False,False,False,False,False,2015
10077,2014-07-26,1C3ANE9L24X016248,0.0,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,1,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,0.000000,False,False,False,False,False,False,2014
103323,2014-10-20,1C8FYN8O63T585854,107950.0,2,3,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2192.0,10,NaN,NaN,2,NaN,NaN,0 days,NaT,NaN,NaT,NaT,NaT,NaN,NaN,NaN,53 days 00:00:00,2.506427e+08,NaN,NaN,0.000000,False,False,False,False,False,False,2014
108984,2014-12-12,1C8FYN8O63T585854,110142.0,2,3,2,1,0,0.0,1.0,1.0,1.0,1.0,1.0,2192.0,636.0,8,10.0,NaN,2,2.0,NaN,53 days,53 days,NaN,NaT,NaT,NaT,NaT,NaN,NaN,52 days 00:00:00,7.282570e+07,2.506427e+08,NaN,0.000000,False,True,True,True,True,True,2014
112403,2015-02-02,1C8FYN8O63T585854,110778.0,2,3,2,1,0,0.0,1.0,1.0,2.0,2.0,2.0,636.0,587.0,14,8.0,10.0,6,2.0,2.0,105 days,52 days,NaN,105 days 00:00:00,NaT,NaT,53 days 00:00:00,NaT,NaN,23 days 00:00:00,5.437718e+07,7.282570e+07,2.506427e+08,1.000000,False,True,True,True,True,True,2015
114316,2015-02-25,1C8FYN8O63T585854,111365.0,2,3,2,1,0,1.0,1.0,2.0,2.0,3.0,3.0,587.0,1565.0,4,14.0,8.0,2,6.0,2.0,128 days,23 days,NaN,75 days 00:00:00,128 days 00:00:00,NaT,52 days 00:00:00,53 days 00:00:00,NaT,117 days 00:00:00,8.299390e+07,5.437718e+07,7.282570e+07,0.051665,True,True,True,True,True,True,2015
125322,2015-06-22,1C8FYN8O63T585854,112930.0,2,3,2,1,0,0.0,0.0,0.0,1.0,2.0,2.0,1565.0,8010.0,5,4.0,14.0,2,2.0,6.0,245 days,117 days,NaN,140 days 00:00:00,192 days 00:00:00,245 days 00:00:00,23 days 00:00:00,52 days 00:00:00,53 days 00:00:00,354 days 00:00:00,1.740418e+08,8.299390e+07,5.437718e+07,-0.017250,False,False,False,True,True,True,2015
159899,2016-06-10,1C8FYN8O63T585854,120940.0,2,3,2,1,0,0.0,0.0,0.0,0.0,0.0,0.0,8010.0,1138.0,10,5.0,4.0,2,2.0,2.0,599 days,354 days,NaN,471 days 00:00:00,494 days 00:00:00,546 days 00:00:00,117 days 00:00:00,23 days 00:00:00,52 days 00:00:00,123 days 00:00:00,1.449721e+08,1.740418e+08,8.299390e+07,-0.007385,False,False,False,False,False,False,2016
171613,2016-10-11,1C8FYN8O63T585854,122078.0,2,3,2,1,0,0.0,0.0,0.0,0.0,1.0,1.0,1138.0,4162.0,15,10.0,5.0,3,2.0,2.0,722 days,123 days,NaN,477 days 00:00:00,594 days 00:00:00,617 days 00:00:00,354 days 00:00:00,117 days 00:00:00,23 days 00:00:00,183 days 00:00:00,3.492072e+08,1.449721e+08,1.740418e+08,-0.004513,False,False,False,False,True,True,2016


In [214]:
"number of group: {} [{}/{} ({:0f}%)]".format(3, 3, 8, 100* 3/8)

'number of group: 3 [3/8 (37.500000%)]'

In [193]:
# test code
# create feature: time distance from last coming
time_shift_forward = pd.concat([pd.Series(datetime(2010, 1, 1)), tmp['Auftragsdatum'][:-1]])
time_shift_forward.index = tmp.index
tmp['time_distance_from_last'] = tmp['Auftragsdatum'] - time_shift_forward
# create feature: time distance to next coming
time_distance_to_next = pd.concat([tmp['time_distance_from_last'][1:], pd.Series(np.NaN)])
time_distance_to_next.index = tmp.index
tmp['time_distance_to_next'] = time_distance_to_next
# reset the first value and last value
tmp.iloc[0, tmp.columns.get_loc('time_distance_from_last')] = np.NaN
tmp.iloc[-1, tmp.columns.get_loc('time_distance_to_next')] = np.NaN
# create feature: km stand distance from last coming
km_shift_forward = pd.concat([pd.Series(0.0), tmp['KM-Stand'][:-1]])
km_shift_forward.index = tmp.index
tmp['km_distance_from_last'] = tmp['KM-Stand'] -km_shift_forward
# create feature: km stand distance to next coming
km_distance_to_next = pd.concat([tmp['km_distance_from_last'][1:], pd.Series(0.0)])
km_distance_to_next.index = tmp.index
tmp['km_distance_to_next'] = km_distance_to_next
# reset the first value and last value
tmp.iloc[0, tmp.columns.get_loc('km_distance_from_last')] = np.NaN
tmp.iloc[-1, tmp.columns.get_loc('km_distance_to_next')] = np.NaN
# create feature: frequent in each year
year_candidates = ['2014', '2015', '2016', '2017', '2018']
for year in year_candidates:
    tmp['frequent_in_'+year] = len(tmp[tmp['year'] == year])
# create feature: time shift for num_act
num_act_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_act'][:-1]])
num_act_shift_forward.index = tmp.index
tmp['num_act_shift1'] = num_act_shift_forward
# create feature: time shift for num_teile
num_teile_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['num_teile'][:-1]])
num_teile_shift_forward.index = tmp.index
tmp['num_teile_shift1'] = num_teile_shift_forward
# create feature: time shift for total aw
total_aw_shift_forward  = pd.concat([pd.Series(np.NaN), tmp['total AW'][:-1]])
total_aw_shift_forward.index = tmp.index
tmp['total_aw_shift1'] = total_aw_shift_forward
if len(tmp) > 2:
    num_act_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_act'][:-2]])
    num_act_shift_forward2.index = tmp.index
    tmp['num_act_shift2'] = num_act_shift_forward2
    num_teile_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['num_teile'][:-2]])
    num_teile_shift_forward2.index = tmp.index
    tmp['num_teile_shift2'] = num_teile_shift_forward2
    total_aw_shift_forward2 = pd.concat([pd.Series([np.NaN, np.NaN]), tmp['total AW'][:-2]])
    total_aw_shift_forward2.index = tmp.index
    tmp['total_aw_shift2'] = total_aw_shift_forward2
else:
    tmp['num_act_shift2'] = np.NaN
    tmp['num_teile_shift2'] = np.NaN
    tmp['total_aw_shift2'] = np.NaN
# create feature: repair age in year or in month
tmp['first_date'] = tmp.iloc[0, tmp.columns.get_loc('Auftragsdatum')]
tmp['repair_age'] = tmp['Auftragsdatum'] - tmp['first_date']
tmp = tmp.drop('first_date', axis=1)
# create feature: repair frequent in last X month
time_distance_shift1 = pd.concat([pd.Series(np.NaN), tmp['time_distance_from_last'][:-1]])
time_distance_shift1.index = tmp.index
tmp['time_distance_shift1'] = time_distance_shift1
time_distance_shift2 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift1'][:-1]])
time_distance_shift2.index = tmp.index
tmp['time_distance_shift2'] = time_distance_shift2
time_distance_shift3 = pd.concat([pd.Series(np.NaN), tmp['time_distance_shift2'][:-1]])
time_distance_shift3.index = tmp.index
tmp['time_distance_shift3'] = time_distance_shift3
tmp['time_distance_from_last2'] = tmp['time_distance_from_last'] + tmp['time_distance_shift1']
tmp['time_distance_from_last3'] = tmp['time_distance_from_last2'] + tmp['time_distance_shift2']
tmp['time_distance_from_last4'] = tmp['time_distance_from_last3'] + tmp['time_distance_shift3']
def get_frequent(x, d = 30):
    if x['time_distance_from_last4'] < timedelta(days = d):
        return 4
    elif x['time_distance_from_last3'] < timedelta(days = d):
        return 3
    elif x['time_distance_from_last2'] < timedelta(days = d):
        return 2
    elif x['time_distance_from_last'] < timedelta(days = d):
        return 1
    else:
        return 0
tmp['frequent_in_last_1_month'] = tmp.apply(get_frequent, axis=1, args = [1*30])
tmp['frequent_in_last_2_month'] = tmp.apply(get_frequent, axis=1, args = [2*30])
tmp['frequent_in_last_3_month'] = tmp.apply(get_frequent, axis=1, args = [3*30])
tmp['frequent_in_last_4_month'] = tmp.apply(get_frequent, axis=1, args = [4*30])
tmp['frequent_in_last_5_month'] = tmp.apply(get_frequent, axis=1, args = [5*30])
tmp['frequent_in_last_6_month'] = tmp.apply(get_frequent, axis=1, args = [6*30])
# create feature: weather come in last X month
tmp['weather_come_in_last_1_month'] = tmp['time_distance_from_last'] < timedelta(days = 1*30)
tmp['weather_come_in_last_2_month'] = tmp['time_distance_from_last'] < timedelta(days = 2*30)
tmp['weather_come_in_last_3_month'] = tmp['time_distance_from_last'] < timedelta(days = 3*30)
tmp['weather_come_in_last_4_month'] = tmp['time_distance_from_last'] < timedelta(days = 4*30)
tmp['weather_come_in_last_5_month'] = tmp['time_distance_from_last'] < timedelta(days = 5*30)
tmp['weather_come_in_last_6_month'] = tmp['time_distance_from_last'] < timedelta(days = 6*30)
# create feature: trend of frequent in the past
def get_trend(x):
    xs = x[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']].tolist()
    xs = [x.days for x in xs if not pd.isnull(x)]
    if len(xs) < 2:
        return 0
    ys = range(1, len(xs) + 1)
    xm = sum(xs)/len(xs)
    ym = sum(ys)/len(ys)
    xy = [x*y for x, y in zip(xs, ys)]
    xx = [x*y for x, y in zip(xs, xs)]
    up = sum(xy) - len(xs)*xm*ym
    down = sum(xx) - len(xs)*xm*xm
    if down == 0:
        return 0
    return up*1.0/down
tmp['trend_of_frequent_in_the_past'] = tmp.apply(get_trend, axis=1)
# create feature: weather come in X month again

In [188]:
def get_trend(x):
    xs = x[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']].tolist()
    xs = [x.days for x in xs if not pd.isnull(x)]
    if len(xs) < 2:
        return 0
    ys = range(1, len(xs) + 1)
    xm = sum(xs)/len(xs)
    ym = sum(ys)/len(ys)
    xy = [x*y for x, y in zip(xs, ys)]
    xx = [x*y for x, y in zip(xs, xs)]
    up = sum(xy) - len(xs)*xm*ym
    down = sum(xx) - len(xs)*xm*xm
    if down == 0:
        return 0
    return up*1.0/down

tmp.apply(get_trend, axis=1)
    #tmp[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']].iloc[1].isna()

103323    0.000000
108984    0.000000
112403    1.000000
114316    0.051665
125322   -0.017250
159899   -0.007385
171613   -0.004513
193678    0.000450
dtype: float64

In [189]:
tmp[['time_distance_from_last', 'time_distance_shift1', 'time_distance_shift2', 'time_distance_shift3']]

,time_distance_from_last,time_distance_shift1,time_distance_shift2,time_distance_shift3
103323,NaT,NaN,NaN,NaN
108984,53 days,NaT,NaN,NaN
112403,52 days,53 days 00:00:00,NaT,NaN
114316,23 days,52 days 00:00:00,53 days 00:00:00,NaT
125322,117 days,23 days 00:00:00,52 days 00:00:00,53 days 00:00:00
159899,354 days,117 days 00:00:00,23 days 00:00:00,52 days 00:00:00
171613,123 days,354 days 00:00:00,117 days 00:00:00,23 days 00:00:00
193678,183 days,123 days 00:00:00,354 days 00:00:00,117 days 00:00:00


In [148]:
a

,time_distance_from_last
103323,NaT
108984,53 days
112403,52 days
114316,23 days
125322,117 days
159899,354 days
171613,123 days
193678,183 days


In [120]:
def get_frequent(x, d = 30):
    if x['time_distance_from_last4'] < timedelta(days = d):
        return 4
    elif x['time_distance_from_last3'] < timedelta(days = d):
        return 3
    elif x['time_distance_from_last2'] < timedelta(days = d):
        return 2
    elif x['time_distance_from_last'] < timedelta(days = d):
        return 1
    else:
        return 0

tmp.apply(get_frequent, axis=1, args = [200])

103323    0
108984    1
112403    2
114316    3
125322    3
159899    0
171613    1
193678    1
dtype: int64

In [118]:
tmp

,Fahrgestellnummer,Auftragsdatum,KM-Stand,num_act,total AW,num_teile,year,repair_age,time_distance_from_last,time_distance_to_next,km_distance_from_last,km_distance_to_next,frequent_in_2014,frequent_in_2015,frequent_in_2016,frequent_in_2017,frequent_in_2018,num_act_shift1,num_teile_shift1,total_aw_shift1,num_act_shift2,num_teile_shift2,total_aw_shift2,time_distance_shift1,time_distance_shift2,time_distance_shift3,time_distance_from_last2,time_distance_from_last3,time_distance_from_last4
103323,1C8FYN8O63T585854,2014-10-20,107950.0,10,250642709.0,2,2014,0 days,NaT,53 days 00:00:00,NaN,2192.0,2,3,2,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
108984,1C8FYN8O63T585854,2014-12-12,110142.0,8,72825699.0,2,2014,53 days,53 days,52 days 00:00:00,2192.0,636.0,2,3,2,1,0,10.0,2.0,250642709.0,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaT,NaT
112403,1C8FYN8O63T585854,2015-02-02,110778.0,14,54377178.0,6,2015,105 days,52 days,23 days 00:00:00,636.0,587.0,2,3,2,1,0,8.0,2.0,72825699.0,10.0,2.0,250642709.0,53 days 00:00:00,NaT,NaN,105 days,NaT,NaT
114316,1C8FYN8O63T585854,2015-02-25,111365.0,4,82993900.0,2,2015,128 days,23 days,117 days 00:00:00,587.0,1565.0,2,3,2,1,0,14.0,6.0,54377178.0,8.0,2.0,72825699.0,52 days 00:00:00,53 days 00:00:00,NaT,75 days,128 days,NaT
125322,1C8FYN8O63T585854,2015-06-22,112930.0,5,174041850.0,2,2015,245 days,117 days,354 days 00:00:00,1565.0,8010.0,2,3,2,1,0,4.0,2.0,82993900.0,14.0,6.0,54377178.0,23 days 00:00:00,52 days 00:00:00,53 days 00:00:00,140 days,192 days,245 days
159899,1C8FYN8O63T585854,2016-06-10,120940.0,10,144972106.0,2,2016,599 days,354 days,123 days 00:00:00,8010.0,1138.0,2,3,2,1,0,5.0,2.0,174041850.0,4.0,2.0,82993900.0,117 days 00:00:00,23 days 00:00:00,52 days 00:00:00,471 days,494 days,546 days
171613,1C8FYN8O63T585854,2016-10-11,122078.0,15,349207211.0,3,2016,722 days,123 days,183 days 00:00:00,1138.0,4162.0,2,3,2,1,0,10.0,2.0,144972106.0,5.0,2.0,174041850.0,354 days 00:00:00,117 days 00:00:00,23 days 00:00:00,477 days,594 days,617 days
193678,1C8FYN8O63T585854,2017-04-12,126240.0,4,98444554.0,2,2017,905 days,183 days,NaN,4162.0,NaN,2,3,2,1,0,15.0,3.0,349207211.0,10.0,2.0,144972106.0,123 days 00:00:00,354 days 00:00:00,117 days 00:00:00,306 days,660 days,777 days


In [113]:
tmp['time_distance_from_last2'] = tmp['time_distance_from_last'] + tmp['time_distance_shift1']

In [100]:
tmp['time_distance_from_last'] > timedelta(days = 60)

103323    False
108984    False
112403    False
114316    False
125322     True
159899     True
171613     True
193678     True
Name: time_distance_from_last, dtype: bool

In [86]:
df2

,Fahrgestellnummer,Auftragsdatum,KM-Stand,num_act,total AW,num_teile,year,time_distance_from_last,time_distance_to_next,km_distance_from_last,km_distance_to_next,frequent_in_2014,frequent_in_2015,frequent_in_2016,frequent_in_2017,frequent_in_2018,num_act_shift1,num_teile_shift1,total_aw_shift1,num_act_shift2,num_teile_shift2,total_aw_shift2
103323,1C8FYN8O63T585854,2014-10-20,107950.0,10,2.506427e+08,2,2014,NaT,53 days 00:00:00,NaN,2192.0,2,3,2,1,0,NaN,NaN,NaN,NaN,NaN,NaN
108984,1C8FYN8O63T585854,2014-12-12,110142.0,8,7.282570e+07,2,2014,53 days,52 days 00:00:00,2192.0,636.0,2,3,2,1,0,10.0,2.0,2.506427e+08,NaN,NaN,NaN
112403,1C8FYN8O63T585854,2015-02-02,110778.0,14,5.437718e+07,6,2015,52 days,23 days 00:00:00,636.0,587.0,2,3,2,1,0,8.0,2.0,7.282570e+07,10.0,2.0,2.506427e+08
114316,1C8FYN8O63T585854,2015-02-25,111365.0,4,8.299390e+07,2,2015,23 days,117 days 00:00:00,587.0,1565.0,2,3,2,1,0,14.0,6.0,5.437718e+07,8.0,2.0,7.282570e+07
125322,1C8FYN8O63T585854,2015-06-22,112930.0,5,1.740418e+08,2,2015,117 days,354 days 00:00:00,1565.0,8010.0,2,3,2,1,0,4.0,2.0,8.299390e+07,14.0,6.0,5.437718e+07
159899,1C8FYN8O63T585854,2016-06-10,120940.0,10,1.449721e+08,2,2016,354 days,123 days 00:00:00,8010.0,1138.0,2,3,2,1,0,5.0,2.0,1.740418e+08,4.0,2.0,8.299390e+07
171613,1C8FYN8O63T585854,2016-10-11,122078.0,15,3.492072e+08,3,2016,123 days,183 days 00:00:00,1138.0,4162.0,2,3,2,1,0,10.0,2.0,1.449721e+08,5.0,2.0,1.740418e+08
193678,1C8FYN8O63T585854,2017-04-12,126240.0,4,9.844455e+07,2,2017,183 days,NaN,4162.0,NaN,2,3,2,1,0,15.0,3.0,3.492072e+08,10.0,2.0,1.449721e+08
82835,1G0R86E41CU106188,2014-01-02,13415.0,16,6.008080e+05,4,2014,NaT,287 days 00:00:00,NaN,10856.0,2,3,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
101439,1G0R86E41CU106188,2014-10-16,24271.0,7,6.009330e+05,2,2014,287 days,146 days 00:00:00,10856.0,8187.0,2,3,0,0,0,16.0,4.0,6.008080e+05,NaN,NaN,NaN


In [23]:
#### It takes too long to run this programm, try other methods instead. ####

# groupby fahrgestellsnummer and compute the date distance between the nearby items
# the condition of the operate below is: we sort the data with 'Fahrgestellnumm' and 'Auftragsdatum'
tmp = df.copy()
td = tmp[['Auftragsdatum', 'Fahrgestellnummer']]
print('To calculate the date distance')
#td['Auftragsdatum'] = pd.to_datetime(td['Auftragsdatum'])
#td = td.drop_duplicates() # drop duplicate
td['Auftragsdatum1'] = pd.to_datetime(td['Auftragsdatum'], dayfirst= True) # transfer to datetime
td = td.sort_values(['Fahrgestellnummer', 'Auftragsdatum1']) # sort with Fahrgestellnummer and datum
td = td.reset_index(drop= True) # shift the data so that we can compute the distance
td['Auftragsdatum2'] = pd.concat([pd.Series(datetime(2010,1,1)), td['Auftragsdatum1'][:-1]],ignore_index=True)
#td['Auftragsdatum2'] = pd.concat([td['Auftragsdatum'][1:], pd.Series('0.0.0')],ignore_index=True)
# compute the date distance
#td['date_dis'] = pd.to_datetime(td['Auftragsdatum']) - pd.to_datetime(td['Auftragsdatum2'])
td['date_dis'] = td['Auftragsdatum1'] - td['Auftragsdatum2']
#td = td[['Auftragsdatum', 'Fahrgestellnummer', 'date_dis']]
print('To delete the first data in the Fahrgestellnummer group')
# groupby Fahrgestellnummer and sort by Auftragsdatum and set the first distance to na
out = []
for name, groupe in td.groupby('Fahrgestellnummer', as_index = False):
    sorted_g = groupe.sort_values('Auftragsdatum1')
    sorted_g['date_dis'].iloc[0] = np.NaN
    out.append(sorted_g)
td = pd.concat(out)
# merge back to the
tmp = pd.merge(df, td, how = 'left', on = ['Fahrgestellnummer', 'Auftragsdatum'])

To calculate the date distance


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


To delete the first data in the Fahrgestellnummer group


/Users/ihuangyiran/anaconda2/envs/data_mining/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


## extract feature: trend of frequent in each year
#toDO

In [24]:
df2 = tmp

In [29]:
df

,Auftragsnummer,total AW,Markencode,Lagerortcode,Auftragsdatum,Adressanredecode,Motorcode,Fahrzeugmodellnummer,Modell,Typ,Getriebecode,Gewicht,Leistung (KW),Fahrgestellnummer,Gruppe-Nr,year,month,day
0,AWSAU195981,1.526062e+09,VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",13.01.2014,Firma,CRBC,5G131X,Golf VII 2.0 TDI CL BMT,X0A,PFL,"0,00",110.0,WVWZZZAUZDP084627,3,2014,01,13
1,AWSAU195981,1.526062e+09,VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",13.01.2014,Firma,CRBC,5G131X,Golf VII 2.0 TDI CL BMT,X0A,PFL,"0,00",110.0,WVWZZZAUZDP084627,0,2014,01,13
2,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,0,2014,01,16
3,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,0,2014,01,16
4,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,9,2014,01,16
5,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,0,2014,01,16
6,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,0,2014,01,16
7,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,0,2014,01,16
8,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,1,2014,01,16
9,AWSAU198256,6.398138e+08,AUDI,"Weeber Leonberg Audi (Glemseckstraße 39, 71229...",16.01.2014,AN,CJC,8K50SC,Audi A4 Attraction Avant 2.0 T,X0A,MVS,"0,00",0.0,WAUZZZ8K0DA098516,9,2014,01,16


# !!!!!habe ich KM-Stand und erst-zulassung-datum vergessen

## extract feature, weather a customer will come in X months for special Gruppe-Nr
here we use the Eletrik, when we use this data to train the model directly, the result may be wrong because of the data balancy.<br>
There are two types of features:
- base on Auftrag, like the total AW-Nr, KM-stand
- base on Fahrgestellnummer, like the frequency(??), long traveler.

In [33]:
df.head()

,Auftragsnummer,total AW,Markencode,Lagerortcode,Adressanredecode,Motorcode,Fahrzeugmodellnummer,Modell,Typ,Getriebecode,Gewicht,Leistung (KW),Fahrgestellnummer
0,A103K,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A77KW,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AWSAU000839,2.168334e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AWSAU195981,1.526062e+09,VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",Firma,CRBC,5G131X,Golf VII 2.0 TDI CL BMT,X0A,PFL,"0,00",110.0,WVWZZZAUZDP084627
4,AWSAU195981,1.526062e+09,VW,"Weeber Leonberg VW (Glemseckstraße 49, 71229 L...",Firma,CRBC,5G131X,Golf VII 2.0 TDI CL BMT,X0A,PFL,"0,00",110.0,WVWZZZAUZDP084627


In [25]:
def check_to_1(df, atta, attb):
    """
    input:
        df, Dataframe:
            the data frame
        atta, string:
            first attribute
        attb, string:
            second attribute
    output:
        out boolean:
            if for an a has more than one b, return True, otherwise return False
    """
    df = df[[atta, attb]]
    df.to_csv('/tmp/tmp.csv', sep = ';', index = False)
    abdict = ABDict('/tmp/tmp.csv', header = True, sep = ';')
    dic = abdict.dic
    flag = 0
    for i,j in dic.items():
        if len(j) > 1:
            print(str(i) + '\t' + str(j))

In [24]:
# 转化为auftrag table， 但是这次合并的是Teile-Nr项
# 给的数据的每一行都是一个维修项，初衷是，把属于同一个auftrag的维修项合并到一起，看一下，在同一个Auftrag中，经常一起修的是那些内容

def toAuftragTable(df, att, auftn, clean = True):
    """
    input:
        df, DataFrame:
            the dataframe
        att, string:
            the column name of the target attribute
        auftn, string:
            the column name of the aftragsnummer attribute
        clean:
            when true, drop the null item in auftn attribute.
    output:
        df_g, DataFrame:
            dataframe contrains two columns auftn and att
            type of item in att is string, separate with ';'
    """
    # assert: make sure the type of the attributes inputted
    
    # extract the att and date columns
    df = df[[att, auftn]]
    # set type to object
    #df[att] = df[att].astype('object')
    #df[auftn] = df[auftn].astype('object')
    # if clean is True, drop the fake data, like the null data
    if clean:
        print("Falls Null date exist, drop these dates directly")
        #df = df.drop(df[df[att].isnull()].index)
        df = df.drop(df[df[auftn].isnull()].index)
    # group and sum 
    df_g = df.groupby([auftn], as_index = False).apply(agg)
    return df_g

# apply 只能对单行进行处理，而不是对整个分组进行处理，所以估计应该把axis换成1，比较好
def agg(x):
    # 是否用‘ ’分隔会比较好，这样就不用对初始的属性，
    # x 在这里是dataframe？？？
    #x = [str(i) for i in x]
    x = x.apply(lambda x: ';'.join(set([str(i) for i in x])), axis = 0)
    #x = x.apply(lambda x: ' '.join(set(x)), axis = 0)
    #print(x.columns.values)
    return x

In [22]:
class ABDict:
    """
    make sure that only two columns in the df, otherwise only the first two columns will be used
    用于两列数据互补空值
    """
    def __init__(self, path, header = True, sep = ';'):
        self.dic = self._load_dict(path, header, sep) 
        self.dic_conv = self._load_dict_conv(path, header, sep)
        print(len(self.dic))
        print(len(self.dic_conv))
        counter = 0
        ########
        # test #
        ########
        #for i in self.dic.keys():
        #    if counter > 1:
        #        break
        #    print(i, self.dic[i])
        #    counter += 1
        #print('schraube' in self.dic_conv.keys())
    
    def AToB(self, a):
        if a in self.dic.keys():
            out = self.dic[a][0]
            if type(out) == list:
                print(out)
            return out
        else:
            #print(a)
            return float('nan')
    
    def BToA(self, b):
        if b in self.dic_conv.keys():
            out = self.dic_conv[b][0]
            #print(out)
            return out
        else:
            #print(b)
            return float('nan')

    def _load_dict(self, path, header, sep):
        dic = {} 
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter ==0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()   # 忘记去换行符了，导致调了一晚上的错
                items = li.split(sep)
                if items[0] not in dic.keys():
                    # if not exit, add new item
                    if items[1] != 'nan':
                        dic[items[0]] = [items[1]]
                elif items[1] not in dic[items[0]]:
                    # if item exit but value not eixt, add the value to the list
                    dic[items[0]].append(items[1])
                counter += 1
        return dic
    
    def _load_dict_conv(self, path, header, sep):
        dic = {}
        with open(path) as fi:
            counter = 0
            for li in fi:
                if header and counter == 0:
                    # drop first line if header is true
                    counter += 1
                    continue
                li = li.strip()
                items = li.split(sep)
                if items[1] not in dic.keys():
                    if items[0] != 'nan':
                        #print(items[0])
                        dic[items[1]] = [items[0]]
                elif items[0] not in dic[items[1]]:
                    dic[items[1]].append(items[0])
                counter += 1
        return dic